In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader,TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [4]:
N_CLASSES = 3
EPOCHS = 3

features,target = make_classification(n_classes=N_CLASSES,
                                        n_informative=9,
                                        n_redundant=0,
                                        n_features=10,
                                        n_samples=1000)

features_train,features_test,target_train,target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=1,
                                                                            random_state=1)

torch.manual_seed(0)
np.random.seed(0)

x_train = torch.from_numpy(features_train).float()
y_train = torch.nn.functional.one_hot(torch.from_numpy(target_train).long(),num_classes=N_CLASSES).float()
x_test = torch.from_numpy(features_test).float()
y_test = torch.nn.functional.one_hot(torch.from_numpy(target_test).long(),num_classes=N_CLASSES).float()

In [5]:
class SimpleneuralNet(nn.Module):
    def __init__(self):
        super(SimpleneuralNet,self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,3),
            torch.nn.Softmax()
        )

    def forward(self,x):
        x = self.sequential(x)
        return x

In [6]:
network = SimpleneuralNet()

criterion = nn.CrossEntropyLoss()
optimizer = RMSprop(network.parameters())

train_data = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_data,batch_size=100,shuffle=True)

for epoch in range(EPOCHS):
    for batch_idx,(data,target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
    print("Epoch:", epoch+1, "\tLoss:", loss.item())

with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output,y_test)
    test_accuracy = (output.round() == y_test).float().mean()
    print("Test Loss:" , test_loss.item(), "\tTest Accuracy:" , test_accuracy.item())
    

d:\New folder\Project\ProjectData\Data Science\datascience\Lib\site-packages\torch\nn\modules\module.py:1775: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch: 1 	Loss: 0.7475630044937134
Epoch: 2 	Loss: 0.7556872367858887
Epoch: 3 	Loss: 0.6515218615531921
Test Loss: 0.5517594814300537 	Test Accuracy: 1.0
